In [ ]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 35.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 26.1 MB/s 

In [ ]:
pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.2 MB/s eta 0:00:00


In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.5 MB/s eta 0:00:00


In [ ]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.llms import CTransformers

llama_model = CTransformers(
    model = "/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q8_0.bin",
    model_type = "llama",
    config = {'max_new_tokens':1000,
              'temperature':0.75,
              'context_length':2000}
)

In [ ]:
# Step 1: load the external data source. In our case, we will load Meta’s “Responsible Use Guide” pdf document.
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
#loader = OnlinePDFLoader("https://ai.meta.com/static-resource/responsible-use-guide/")
loader = PyPDFDirectoryLoader('./data')
documents = loader.load()

# Step 2: Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# Step 3: Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Step 4: Use vector store to store embeddings
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [ ]:
def md(t):
  display(Markdown(t))

In [ ]:
# Query against your own data
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
query = "what is covid 19 treatment in severe cases"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

 According to the text, there are several treatments for COVID-19 in severe cases, including:

* Mild pain relievers
* Cough syrup
* Resting and high amount of fluid intake
* Ventilators to help infected patients breathe and support lung function (although ventilators do not cure COVID-19)
* ECMO (extracorporeal membrane oxygenation), a life-saving technique to support the body during infection.

It is important to note that these treatments are based on early symptoms of COVID-19 and are not necessarily effective for all cases. Additionally, some medications are under investigation and are being tested through clinical trials in the United States and around the world.

In [ ]:
query = "How to Know I have covid 19?"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

 Based on the information provided above, there are several common symptoms of COVID-19 that may help identify an infected person. These include fever, cough, muscle pain, headache, hemoptysis, and diarrhea (Fig. 6). However, it is essential to note that some people may not show any symptoms at all or may only experience mild symptoms. Therefore, if you suspect that you or someone in your vicinity may have COVID-19, please consult a healthcare professional for proper diagnosis and treatment.

In [ ]:
query = input()
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])